## alert data classification solution walk through

In [3]:
# read raw alert data
import json
import pandas as pd
import sys, os, re

alert_df = pd.read_csv('C:/Users/jshay21/alert_classification/data_consolidation_beta/alerts_all.csv', dtype=str, on_bad_lines='skip')
alert_df.fillna('', inplace=True)
alert_df.head(5)

,Unnamed: 0,_id,batch_id,starts_at,ends_at,status,service_name,alert_category,alert_subcategory,severity,alert_name,recorded_at,payload
0,status,68b83470f013e1a62ae22c9b,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:05:19.573,2025-09-03 12:06:19.573,resolved,emua-payment-df-service,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,resolved
1,labels,68b83470f013e1a62ae22c9b,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:05:19.573,2025-09-03 12:06:19.573,resolved,emua-payment-df-service,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,"{'__tenant_id__': 'aks-res-clusters', 'aiops':..."
2,annotations,68b83470f013e1a62ae22c9b,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:05:19.573,2025-09-03 12:06:19.573,resolved,emua-payment-df-service,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,{'description': 'The network:rx_bytes rate for...
3,startsAt,68b83470f013e1a62ae22c9b,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:05:19.573,2025-09-03 12:06:19.573,resolved,emua-payment-df-service,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,2025-09-03T12:05:19.573Z
4,endsAt,68b83470f013e1a62ae22c9b,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:05:19.573,2025-09-03 12:06:19.573,resolved,emua-payment-df-service,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,2025-09-03T12:06:19.573Z


In [8]:
# pivot data as structured format
# Pivot the data to get structured alerts
alert_df = alert_df.rename(columns={
    alert_df.columns[0]: "attribute", 
    alert_df.columns[-1]: "value"
})

id_cols = [c for c in alert_df.columns if c not in ("attribute", "value")]
df_pivoted = alert_df.pivot_table(
    index=id_cols, 
    columns='attribute', 
    values='value', 
    aggfunc='first'
)
df_pivoted.head(5)
print("firing alerts structured")

firing_df = df_pivoted[df_pivoted['status'].str.strip().str.lower() == 'firing']
display(firing_df.head(5))

firing alerts structured


,,,,,,,,,,attribute,annotations,endsAt,fingerprint,labels,startsAt,status
_id,batch_id,starts_at,ends_at,status,service_name,alert_category,alert_subcategory,severity,alert_name,recorded_at,,,,,,
68b83470f013e1a62ae22b40,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:27:19.573,1-01-01 00:00:00.000,firing,aks-escoresnp07-32113162-vmss00005u,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,{'description': 'The network:rx_bytes rate for...,0001-01-01T00:00:00Z,33afbffa4e7d838c,"{'__tenant_id__': 'aks-res-clusters', 'aiops':...",2025-09-03T12:27:19.573Z,firing
68b83470f013e1a62ae22b41,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:27:19.573,1-01-01 00:00:00.000,firing,aks-escoba1np02-13277147-vmss0004sn,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,{'description': 'The network:rx_bytes rate for...,0001-01-01T00:00:00Z,7f3771ace34f6d68,"{'__tenant_id__': 'aks-tru-clusters', 'aiops':...",2025-09-03T12:27:19.573Z,firing
68b83470f013e1a62ae22b42,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:21:19.573,1-01-01 00:00:00.000,firing,aks-escoba1np04-28230666-vmss00015m,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,{'description': 'The network:rx_bytes rate for...,0001-01-01T00:00:00Z,e6ee95214611d53b,"{'__tenant_id__': 'aks-tru-clusters', 'aiops':...",2025-09-03T12:21:19.573Z,firing
68b83470f013e1a62ae22b49,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:20:19.573,1-01-01 00:00:00.000,firing,aks-escoba1np01-28933280-vmss0004k5,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,{'description': 'The network:tx_bytes rate for...,0001-01-01T00:00:00Z,207eca87ec34832f,"{'__tenant_id__': 'aks-tru-clusters', 'aiops':...",2025-09-03T12:20:19.573Z,firing
68b83470f013e1a62ae22b4a,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:19:19.573,1-01-01 00:00:00.000,firing,aks-escoba1np02-13277147-vmss0004sn,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,{'description': 'The network:tx_bytes rate for...,0001-01-01T00:00:00Z,fa16bc1116c043ce,"{'__tenant_id__': 'aks-tru-clusters', 'aiops':...",2025-09-03T12:19:19.573Z,firing


In [12]:
## show metadata extraction

import pandas as pd
import ast

# Load raw CSV data
df_raw = pd.read_csv('C:/Users/jshay21/alert_classification/data_consolidation_beta/alerts_all.csv', dtype=str, on_bad_lines='skip')

# Step 2: Rename columns for pivoting
df_raw = df_raw.rename(columns={
    df_raw.columns[0]: "attribute", 
    df_raw.columns[-1]: "value"
})

# Identify ID columns (all except attribute and value)
id_cols = [c for c in df_raw.columns if c not in ("attribute", "value")]

# Pivot data to get structured alerts (rows = alerts, columns = attributes)
df_pivoted = df_raw.pivot_table(
    index=id_cols, 
    columns='attribute', 
    values='value', 
    aggfunc='first'
)

# Handle conflicting column names before reset_index
index_names = list(df_pivoted.index.names)
conflicting_names = [name for name in index_names if name in df_pivoted.columns]

if conflicting_names:
    df_pivoted = df_pivoted.drop(columns=conflicting_names)

df_pivoted = df_pivoted.reset_index()

# Filter for firing alerts only
if 'status' in df_pivoted.columns:
    firing_df = df_pivoted[df_pivoted['status'].str.strip().str.lower() == 'firing']
else:
    firing_df = df_pivoted

# Convert to list of dictionaries for processing
firing_alerts = firing_df.to_dict('records')

print(f"Loaded {len(firing_alerts)} firing alerts")

# Metadata Extraction
# Parse labels to extract service information
for alert in firing_alerts:
    labels_str = alert.get('labels', '')
    if labels_str:
        try:
            # Parse label string to dictionary
            labels = ast.literal_eval(labels_str)
            
            # Extract service metadata
            alert['service_name'] = labels.get('service_name', '')
            alert['namespace'] = labels.get('namespace', '')
            alert['pod'] = labels.get('pod', '')
            alert['node'] = labels.get('node', '')
            alert['cluster'] = labels.get('cluster', '')
            alert['workload_type'] = labels.get('workload_type', '')
            alert['anomaly_resource_type'] = labels.get('anomaly_resource_type', '')
            alert['alert_category'] = labels.get('alert_category', '')
            alert['alert_subcategory'] = labels.get('alert_subcategory', '')
            alert['platform'] = labels.get('platform', '')
        except (ValueError, SyntaxError, TypeError):
            # Set defaults if parsing fails
            alert['service_name'] = ''
            alert['namespace'] = ''
            # ... etc
    
    # Parse annotations to extract description
    annotations_str = alert.get('annotations', '')
    if annotations_str:
        try:
            annotations = ast.literal_eval(annotations_str)
            alert['description'] = annotations.get('description', '')
        except (ValueError, SyntaxError, TypeError):
            alert['description'] = ''
    
    # Parse temporal information
    starts_at = alert.get('startsAt') or alert.get('starts_at', '')
    if starts_at:
        alert['start_datetime'] = pd.to_datetime(starts_at)
        alert['start_timestamp'] = alert['start_datetime'].timestamp()
    else:
        alert['start_datetime'] = None
        alert['start_timestamp'] = 0

# Show sample alert with extracted metadata
print("\nSample alert with extracted metadata:")
print(f"Service: {firing_alerts[0].get('service_name')}")
print(f"Namespace: {firing_alerts[0].get('namespace')}")
print(f"Alert Category: {firing_alerts[0].get('alert_category')}")
print(f"Alert Subcategory: {firing_alerts[0].get('alert_subcategory')}")
print(f"Description: {firing_alerts[0].get('description')}")
print(f"Start Time: {firing_alerts[0].get('start_datetime')}")
display(firing_df.head(2))

Loaded 13960 firing alerts

Sample alert with extracted metadata:
Service: 
Namespace: 
Alert Category: anomaly
Alert Subcategory: Resource
Description: The network:rx_bytes rate for     has fallen below its lower bound or risen above its upper bound.
Start Time: 2025-09-03 12:27:19.573000+00:00

Sample alert with extracted metadata:
Service: 
Namespace: 
Alert Category: anomaly
Alert Subcategory: Resource
Description: The network:rx_bytes rate for     has fallen below its lower bound or risen above its upper bound.
Start Time: 2025-09-03 12:27:19.573000+00:00


attribute,_id,batch_id,starts_at,ends_at,status,service_name,alert_category,alert_subcategory,severity,alert_name,recorded_at,annotations,endsAt,fingerprint,labels,startsAt
3,68b83470f013e1a62ae22b40,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:27:19.573,1-01-01 00:00:00.000,firing,aks-escoresnp07-32113162-vmss00005u,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,{'description': 'The network:rx_bytes rate for...,0001-01-01T00:00:00Z,33afbffa4e7d838c,"{'__tenant_id__': 'aks-res-clusters', 'aiops':...",2025-09-03T12:27:19.573Z
4,68b83470f013e1a62ae22b41,26f667bf-b56d-4949-93c4-c81b8efb5932,2025-09-03 12:27:19.573,1-01-01 00:00:00.000,firing,aks-escoba1np02-13277147-vmss0004sn,anomaly,Resource,warning,ResourceRateAnomaly,2025-09-03 12:28:32.405,{'description': 'The network:rx_bytes rate for...,0001-01-01T00:00:00Z,7f3771ace34f6d68,"{'__tenant_id__': 'aks-tru-clusters', 'aiops':...",2025-09-03T12:27:19.573Z


In [13]:

alerts_with_metadata = pd.DataFrame(firing_alerts)

# Select key columns to display
metadata_columns = [
    'service_name', 'namespace', 'pod', 'cluster', 'node',
    'alert_category', 'alert_subcategory', 'workload_type',
    'description', 'start_datetime', 'severity'
]

# Filter to only existing columns
display_cols = [col for col in metadata_columns if col in alerts_with_metadata.columns]

print(f"\nDisplaying {len(alerts_with_metadata)} firing alerts with extracted metadata")
print(f"Total columns available: {len(alerts_with_metadata.columns)}")
print(f"\nExtracted metadata fields: {display_cols}\n")

# Display sample of alerts with metadata
display(alerts_with_metadata[display_cols].head(3))

# Show metadata extraction summary
print("\n=== Metadata Extraction Summary ===")
print(f"Total alerts: {len(alerts_with_metadata)}")
print(f"Alerts with service_name: {alerts_with_metadata['service_name'].astype(bool).sum()}")
print(f"Alerts with namespace: {alerts_with_metadata['namespace'].astype(bool).sum()}")
print(f"Alerts with alert_category: {alerts_with_metadata['alert_category'].astype(bool).sum()}")
print(f"Alerts with alert_subcategory: {alerts_with_metadata['alert_subcategory'].astype(bool).sum()}")
print(f"\nUnique services: {alerts_with_metadata['service_name'].nunique()}")
print(f"Unique namespaces: {alerts_with_metadata['namespace'].nunique()}")
print(f"Unique alert categories: {alerts_with_metadata['alert_category'].nunique()}")
print(f"Unique alert subcategories: {alerts_with_metadata['alert_subcategory'].nunique()}")


Displaying 13960 firing alerts with extracted metadata
Total columns available: 26

Extracted metadata fields: ['service_name', 'namespace', 'pod', 'cluster', 'node', 'alert_category', 'alert_subcategory', 'workload_type', 'description', 'start_datetime', 'severity']



,service_name,namespace,pod,cluster,node,alert_category,alert_subcategory,workload_type,description,start_datetime,severity
0,,,,esco-aksres-nonprod-westus-cluster-01,aks-escoresnp07-32113162-vmss00005u,anomaly,Resource,,The network:rx_bytes rate for has fallen b...,2025-09-03 12:27:19.573000+00:00,warning
1,,,,esco-aksba1-nonprod-westus-cluster-01,aks-escoba1np02-13277147-vmss0004sn,anomaly,Resource,,The network:rx_bytes rate for has fallen b...,2025-09-03 12:27:19.573000+00:00,warning
2,,,,esco-aksba1-nonprod-westus-cluster-01,aks-escoba1np04-28230666-vmss00015m,anomaly,Resource,,The network:rx_bytes rate for has fallen b...,2025-09-03 12:21:19.573000+00:00,warning



=== Metadata Extraction Summary ===
Total alerts: 13960
Alerts with service_name: 11980
Alerts with namespace: 12304
Alerts with alert_category: 13960
Alerts with alert_subcategory: 13960

Unique services: 871
Unique namespaces: 411
Unique alert categories: 7
Unique alert subcategories: 16


In [15]:
# Complete _load_graph_data method implementation
import json
import networkx as nx
import numpy as np

def load_svc_map(graph_json_path):
    service_graph = nx.DiGraph()
    service_to_graph = {}
    graph_relationships = []
 
    with open(graph_json_path, 'r') as f:
        graph_data = json.load(f)
    
    # Extract nodes and edges
    nodes = graph_data['data'].get('nodes', [])
    edges = graph_data['data'].get('edges', [])
    
    print(f"    Loaded {len(nodes)} nodes and {len(edges)} edges")
    print("    Building service graph from nodes...")
    
    # Build node mapping: id -> service name
    node_map = {}
    for node in nodes:
        node_id = node.get('id')
        node_props = node.get('properties', {})
        service_name = node_props.get('name', '')
        
        if service_name:
            node_map[node_id] = service_name
            
            # Store service information in service_to_graph dictionary
            service_to_graph[service_name] = {
                'graph_name': service_name,
                'properties': node_props,
                'type': node.get('label', ''),
                'environment': node_props.get('environment', ''),
                'namespace': node_props.get('namespace', ''),
                'cluster': node_props.get('cluster', '')
            }
            
            # Add node to NetworkX graph with all properties
            service_graph.add_node(service_name, **node_props)
    
    print(f"    Built {len(service_to_graph)} service mappings")
    
    # Build edges (relationships between services)
    print("    Building edges...")
    for edge in edges:
        source_id = edge.get('source')
        target_id = edge.get('target')
        edge_label = edge.get('label', 'CALLS')
        
        source_name = node_map.get(source_id)
        target_name = node_map.get(target_id)
        
        if source_name and target_name:
            service_graph.add_edge(
                source_name,
                target_name,
                relationship_type=edge_label
            )
    
    print(f" Built graph: {len(service_to_graph)} services, {service_graph.number_of_edges()} edges")
    
    # Convert to relationship format (for compatibility)
    print("    Creating relationship list...")
    for edge in edges:
        source_id = edge.get('source')
        target_id = edge.get('target')
        source_name = node_map.get(source_id)
        target_name = node_map.get(target_id)
        
        if source_name and target_name:
            source_node = next((n for n in nodes if n.get('id') == source_id), None)
            target_node = next((n for n in nodes if n.get('id') == target_id), None)
            
            if source_node and target_node:
                graph_relationships.append({
                    'source_name': source_name,
                    'source_properties': source_node.get('properties', {}),
                    'target_name': target_name,
                    'target_properties': target_node.get('properties', {}),
                    'relationship_type': edge.get('label', 'CALLS')
                })
    
    print(f"    Created {len(graph_relationships)} relationship records")
    
    # Pre-compute centrality metrics
    print("    Computing graph metrics...")
    pagerank_cache = {}
    betweenness_cache = {}
    clustering_coef_cache = {}
    undirected_graph = None
    
    try:
        # PageRank - measures importance based on incoming connections
        pagerank_cache = nx.pagerank(service_graph)
        
        # Betweenness centrality - identifies bridge/hub services
        betweenness_cache = nx.betweenness_centrality(
            service_graph, 
            k=min(100, len(service_graph))
        )
        
        # Cache undirected graph for clustering coefficient calculations
        undirected_graph = service_graph.to_undirected()
        
        # Clustering coefficient - measures how well-connected neighbors are
        print("    Pre-computing clustering coefficients...")
        clustering_dict = nx.clustering(undirected_graph)
        clustering_coef_cache = clustering_dict
        
        print(f"    Computed metrics for {len(pagerank_cache)} services")
        
    except Exception as e:
        print(f"    Warning: Could not compute some graph metrics: {e}")
    
    # Pre-compute all graph features per service
    print("    Pre-computing graph features per service...")
    service_features_cache = {}
    
    for service_name in service_graph.nodes():
        if service_name not in service_features_cache:
            features = {}
            
            # Basic degree metrics
            features['degree_total'] = service_graph.degree(service_name)
            features['in_degree'] = service_graph.in_degree(service_name)
            features['out_degree'] = service_graph.out_degree(service_name)
            
            # Centrality features (from cached computations)
            features['pagerank'] = pagerank_cache.get(service_name, 0) if pagerank_cache else 0
            features['betweenness'] = betweenness_cache.get(service_name, 0) if betweenness_cache else 0
            features['clustering_coef'] = clustering_coef_cache.get(service_name, 0)
            
            # Relationship type counts (upstream and downstream)
            upstream_rels = []
            downstream_rels = []
            
            # Count upstream relationships (predecessors)
            for predecessor in service_graph.predecessors(service_name):
                edge_data = service_graph.get_edge_data(predecessor, service_name)
                rel_type = edge_data.get('relationship_type', '') if edge_data else ''
                upstream_rels.append(rel_type)
            
            # Count downstream relationships (successors)
            for successor in service_graph.successors(service_name):
                edge_data = service_graph.get_edge_data(service_name, successor)
                rel_type = edge_data.get('relationship_type', '') if edge_data else ''
                downstream_rels.append(rel_type)
            
            features['num_upstream'] = len(upstream_rels)
            features['num_downstream'] = len(downstream_rels)
            features['upstream_calls'] = upstream_rels.count('CALLS')
            features['upstream_owns'] = upstream_rels.count('OWNS')
            features['upstream_belongs_to'] = upstream_rels.count('BELONGS_TO')
            features['downstream_calls'] = downstream_rels.count('CALLS')
            features['downstream_owns'] = downstream_rels.count('OWNS')
            features['downstream_belongs_to'] = downstream_rels.count('BELONGS_TO')
            
            # Relationship ratios
            total_rels = len(upstream_rels) + len(downstream_rels)
            if total_rels > 0:
                features['ratio_calls'] = (features['upstream_calls'] + features['downstream_calls']) / total_rels
                features['ratio_owns'] = (features['upstream_owns'] + features['downstream_owns']) / total_rels
                features['ratio_belongs_to'] = (features['upstream_belongs_to'] + features['downstream_belongs_to']) / total_rels
            else:
                features['ratio_calls'] = 0
                features['ratio_owns'] = 0
                features['ratio_belongs_to'] = 0
            
            # Dependency direction (positive = provider, negative = consumer)
            features['dependency_direction'] = features['out_degree'] - features['in_degree']
            
            # Neighborhood features
            neighbors = list(service_graph.predecessors(service_name)) + list(service_graph.successors(service_name))
            if neighbors:
                neighbor_degrees = [service_graph.degree(n) for n in neighbors]
                features['avg_neighbor_degree'] = np.mean(neighbor_degrees)
                features['max_neighbor_degree'] = np.max(neighbor_degrees)
            else:
                features['avg_neighbor_degree'] = 0
                features['max_neighbor_degree'] = 0
            
            service_features_cache[service_name] = features
    
    print(f" Pre-computed features for {len(service_features_cache)} services")
    
    return {
        'service_graph': service_graph,
        'service_to_graph': service_to_graph,
        'graph_relationships': graph_relationships,
        'service_features_cache': service_features_cache,
        'pagerank_cache': pagerank_cache,
        'betweenness_cache': betweenness_cache,
        'clustering_coef_cache': clustering_coef_cache,
        'undirected_graph': undirected_graph
    }

# graph loading
graph_path = 'C:/Users/jshay21/alert_classification/data_consolidation_beta/service_graph_data.json'
graph_data_complete = load_svc_map(graph_path)

    Loaded 17858 nodes and 19176 edges
    Building service graph from nodes...
    Built 16064 service mappings
    Building edges...
 Built graph: 16064 services, 17162 edges
    Creating relationship list...
    Created 19176 relationship records
    Computing graph metrics...
    Created 19176 relationship records
    Computing graph metrics...
    Pre-computing clustering coefficients...
    Pre-computing clustering coefficients...
    Computed metrics for 16064 services
    Pre-computing graph features per service...
    Computed metrics for 16064 services
    Pre-computing graph features per service...
 Pre-computed features for 16064 services
 Pre-computed features for 16064 services


## Graph Data Validation

Let's validate the graph loading logic by checking for inconsistencies.

In [35]:
# Validate graph loading - check for discrepancies
import json

# Load raw JSON to inspect
with open('C:/Users/jshay21/alert_classification/data_consolidation_beta/service_graph_data.json', 'r') as f:
    raw_graph_data = json.load(f)

nodes_raw = raw_graph_data['data'].get('nodes', [])
edges_raw = raw_graph_data['data'].get('edges', [])

print("=== RAW JSON DATA ===")
print(f"Total nodes in JSON: {len(nodes_raw)}")
print(f"Total edges in JSON: {len(edges_raw)}")

# Check for nodes with missing names
nodes_without_names = 0
node_ids_with_names = set()

for node in nodes_raw:
    node_id = node.get('id')
    node_props = node.get('properties', {})
    service_name = node_props.get('name', '')
    
    if service_name:
        node_ids_with_names.add(node_id)
    else:
        nodes_without_names += 1

print(f"\nNodes with valid names: {len(node_ids_with_names)}")
print(f"Nodes WITHOUT names (will be skipped): {nodes_without_names}")

# Check edges
valid_edges = 0
invalid_edges_missing_source = 0
invalid_edges_missing_target = 0

for edge in edges_raw:
    source_id = edge.get('source')
    target_id = edge.get('target')
    
    source_valid = source_id in node_ids_with_names
    target_valid = target_id in node_ids_with_names
    
    if source_valid and target_valid:
        valid_edges += 1
    else:
        if not source_valid:
            invalid_edges_missing_source += 1
        if not target_valid:
            invalid_edges_missing_target += 1

print(f"\n=== EDGE VALIDATION ===")
print(f"Valid edges (both nodes have names): {valid_edges}")
print(f"Invalid edges (source missing name): {invalid_edges_missing_source}")
print(f"Invalid edges (target missing name): {invalid_edges_missing_target}")
print(f"Total invalid edges: {len(edges_raw) - valid_edges}")

# Compare with loaded graph
print(f"\n=== LOADED GRAPH COMPARISON ===")
print(f"Expected valid edges: {valid_edges}")
print(f"Actual edges in service_graph: {service_graph.number_of_edges()}")
print(f"Relationship records created: {len(graph_data_complete['graph_relationships'])}")

print(f"\nDifference: {valid_edges - service_graph.number_of_edges()} edges missing from graph")

# Check for duplicate edges
print(f"\n=== DUPLICATE EDGE ANALYSIS ===")
edge_pairs = set()
duplicate_count = 0

for edge in edges_raw:
    source_id = edge.get('source')
    target_id = edge.get('target')
    
    if source_id in node_ids_with_names and target_id in node_ids_with_names:
        edge_pair = (source_id, target_id)
        if edge_pair in edge_pairs:
            duplicate_count += 1
        else:
            edge_pairs.add(edge_pair)

print(f"Unique edge pairs: {len(edge_pairs)}")
print(f"Duplicate edges in JSON: {duplicate_count}")

=== RAW JSON DATA ===
Total nodes in JSON: 17858
Total edges in JSON: 19176

Nodes with valid names: 17858
Nodes WITHOUT names (will be skipped): 0

=== EDGE VALIDATION ===
Valid edges (both nodes have names): 19176
Invalid edges (source missing name): 0
Invalid edges (target missing name): 0
Total invalid edges: 0

=== LOADED GRAPH COMPARISON ===
Expected valid edges: 19176
Actual edges in service_graph: 17162
Relationship records created: 19176

Difference: 2014 edges missing from graph

=== DUPLICATE EDGE ANALYSIS ===
Unique edge pairs: 19176
Duplicate edges in JSON: 0


## Sample Node & Edge Inspection

Let's look at specific examples to understand the structure.

In [36]:
# Inspect sample nodes and edges
print("=== SAMPLE NODES ===")
print("\nNodes WITH names (first 3):")
for i, node in enumerate(nodes_raw[:20]):
    node_props = node.get('properties', {})
    service_name = node_props.get('name', '')
    if service_name:
        print(f"\nNode {i+1}:")
        print(f"  ID: {node.get('id')}")
        print(f"  Label: {node.get('label', 'N/A')}")
        print(f"  Name: {service_name}")
        print(f"  Namespace: {node_props.get('namespace', 'N/A')}")
        print(f"  Cluster: {node_props.get('cluster', 'N/A')}")
        if i >= 2:
            break

print("\n\nNodes WITHOUT names (first 3):")
count = 0
for i, node in enumerate(nodes_raw):
    node_props = node.get('properties', {})
    service_name = node_props.get('name', '')
    if not service_name:
        print(f"\nNode {i+1}:")
        print(f"  ID: {node.get('id')}")
        print(f"  Label: {node.get('label', 'N/A')}")
        print(f"  Properties: {node_props}")
        count += 1
        if count >= 3:
            break

print("\n\n=== SAMPLE EDGES ===")
print("\nFirst 5 edges:")
for i, edge in enumerate(edges_raw[:5]):
    source_id = edge.get('source')
    target_id = edge.get('target')
    
    # Find source and target names
    source_name = None
    target_name = None
    
    for node in nodes_raw:
        if node.get('id') == source_id:
            source_name = node.get('properties', {}).get('name', 'NO_NAME')
        if node.get('id') == target_id:
            target_name = node.get('properties', {}).get('name', 'NO_NAME')
    
    print(f"\nEdge {i+1}:")
    print(f"  Source ID: {source_id} -> Name: {source_name}")
    print(f"  Target ID: {target_id} -> Name: {target_name}")
    print(f"  Relationship: {edge.get('label', 'N/A')}")
    print(f"  Valid: {source_name != 'NO_NAME' and target_name != 'NO_NAME'}")

=== SAMPLE NODES ===

Nodes WITH names (first 3):

Node 1:
  ID: 0
  Label: Service
  Name: ocpe-storecloudconfig
  Namespace: ocpe-qa2
  Cluster: 

Node 2:
  ID: 1
  Label: Service
  Name: ocmo-3pmtoorchestrator
  Namespace: ocmo-perf1
  Cluster: 

Node 3:
  ID: 2
  Label: Service
  Name: menfpt-calendar-service
  Namespace: menfpt-qa1
  Cluster: 


Nodes WITHOUT names (first 3):


=== SAMPLE EDGES ===

First 5 edges:

Edge 1:
  Source ID: 0 -> Name: ocpe-storecloudconfig
  Target ID: 2348 -> Name: epe_acceptance
  Relationship: CALLS
  Valid: True

Edge 2:
  Source ID: 1 -> Name: ocmo-3pmtoorchestrator
  Target ID: 2356 -> Name: order_info
  Relationship: CALLS
  Valid: True

Edge 3:
  Source ID: 1 -> Name: ocmo-3pmtoorchestrator
  Target ID: 16368 -> Name: osfm-order-create
  Relationship: CALLS
  Valid: True

Edge 4:
  Source ID: 2 -> Name: menfpt-calendar-service
  Target ID: 3 -> Name: menfpt_qa1
  Relationship: CALLS
  Valid: True

Edge 5:
  Source ID: 4 -> Name: menfpt-forecast

## Root Cause Analysis

Identify why edges are being lost during graph construction.

In [37]:
# Root cause: Check if NetworkX is de-duplicating edges or if there's a mapping issue

print("=== ROOT CAUSE ANALYSIS ===\n")

# Build node_map again to trace the issue
node_map_test = {}
services_added = 0
nodes_skipped = 0

for node in nodes_raw:
    node_id = node.get('id')
    node_props = node.get('properties', {})
    service_name = node_props.get('name', '')
    
    if service_name:
        if node_id in node_map_test:
            print(f"WARNING: Duplicate node ID detected: {node_id} -> {service_name}")
        node_map_test[node_id] = service_name
        services_added += 1
    else:
        nodes_skipped += 1

print(f"Node Map built: {len(node_map_test)} mappings")
print(f"Services added: {services_added}")
print(f"Nodes skipped (no name): {nodes_skipped}")

# Now check edge addition
print(f"\n=== EDGE ADDITION SIMULATION ===")

edges_attempted = 0
edges_both_valid = 0
edges_source_missing = 0
edges_target_missing = 0
edge_pairs_unique = set()

for edge in edges_raw:
    source_id = edge.get('source')
    target_id = edge.get('target')
    
    edges_attempted += 1
    
    source_name = node_map_test.get(source_id)
    target_name = node_map_test.get(target_id)
    
    if source_name and target_name:
        edges_both_valid += 1
        edge_pairs_unique.add((source_name, target_name))
    else:
        if not source_name:
            edges_source_missing += 1
        if not target_name:
            edges_target_missing += 1

print(f"Total edges attempted: {edges_attempted}")
print(f"Edges with both valid nodes: {edges_both_valid}")
print(f"Edges with missing source: {edges_source_missing}")
print(f"Edges with missing target: {edges_target_missing}")
print(f"Unique edge pairs (source->target): {len(edge_pairs_unique)}")

print(f"\n=== CONCLUSION ===")
print(f"JSON edges: {len(edges_raw)}")
print(f"Valid edges: {edges_both_valid}")
print(f"Unique pairs: {len(edge_pairs_unique)}")
print(f"NetworkX graph edges: {service_graph.number_of_edges()}")
print(f"Relationship records: {len(graph_data_complete['graph_relationships'])}")

if len(edge_pairs_unique) != service_graph.number_of_edges():
    print(f"\n⚠️ MISMATCH: Unique pairs ({len(edge_pairs_unique)}) != Graph edges ({service_graph.number_of_edges()})")
    print("Possible causes:")
    print("  1. Duplicate edges in JSON with same (source, target) pair")
    print("  2. Self-loops being filtered")
    print("  3. Issues with edge addition logic")
else:
    print(f"\n✓ Graph edges match unique edge pairs")

if edges_both_valid != len(graph_data_complete['graph_relationships']):
    print(f"\n⚠️ MISMATCH: Valid edges ({edges_both_valid}) != Relationship records ({len(graph_data_complete['graph_relationships'])})")
else:
    print(f"\n✓ Relationship records match valid edges")

=== ROOT CAUSE ANALYSIS ===

Node Map built: 17858 mappings
Services added: 17858
Nodes skipped (no name): 0

=== EDGE ADDITION SIMULATION ===
Total edges attempted: 19176
Edges with both valid nodes: 19176
Edges with missing source: 0
Edges with missing target: 0
Unique edge pairs (source->target): 17162

=== CONCLUSION ===
JSON edges: 19176
Valid edges: 19176
Unique pairs: 17162
NetworkX graph edges: 17162
Relationship records: 19176

✓ Graph edges match unique edge pairs

✓ Relationship records match valid edges


## Check for Self-Loops and Multi-Edges

NetworkX DiGraph allows multiple edges between same nodes but stores only one by default.

In [38]:
# Check for duplicate edges and self-loops
print("=== DUPLICATE EDGES & SELF-LOOPS ANALYSIS ===\n")

edge_counter = {}
self_loops = 0
multi_edges = []

for edge in edges_raw:
    source_id = edge.get('source')
    target_id = edge.get('target')
    
    source_name = node_map_test.get(source_id)
    target_name = node_map_test.get(target_id)
    
    if source_name and target_name:
        # Check for self-loops
        if source_name == target_name:
            self_loops += 1
            continue
        
        # Count edge occurrences
        edge_key = (source_name, target_name)
        if edge_key not in edge_counter:
            edge_counter[edge_key] = 0
        edge_counter[edge_key] += 1

# Find multi-edges (same source->target appearing multiple times)
for edge_key, count in edge_counter.items():
    if count > 1:
        multi_edges.append((edge_key, count))

print(f"Self-loops detected: {self_loops}")
print(f"Unique edge pairs (excluding self-loops): {len(edge_counter)}")
print(f"Multi-edges (duplicate pairs): {len(multi_edges)}")

if multi_edges:
    print(f"\nTop 10 duplicate edge pairs:")
    multi_edges.sort(key=lambda x: x[1], reverse=True)
    for (source, target), count in multi_edges[:10]:
        print(f"  {source} -> {target}: {count} times")

# Calculate expected edges
expected_edges = len(edge_counter)  # Unique pairs only
print(f"\n=== EXPECTED vs ACTUAL ===")
print(f"Expected unique edges (no self-loops, no duplicates): {expected_edges}")
print(f"Actual NetworkX graph edges: {service_graph.number_of_edges()}")
print(f"Difference: {abs(expected_edges - service_graph.number_of_edges())}")

if expected_edges == service_graph.number_of_edges():
    print("\n✓ CORRECT: NetworkX correctly de-duplicated edges")
    print("  - Self-loops ignored")
    print("  - Duplicate edges merged (keeps last one)")
else:
    print(f"\n⚠️ MISMATCH: Still {abs(expected_edges - service_graph.number_of_edges())} edges missing")

=== DUPLICATE EDGES & SELF-LOOPS ANALYSIS ===

Self-loops detected: 59
Unique edge pairs (excluding self-loops): 17123
Multi-edges (duplicate pairs): 873

Top 10 duplicate edge pairs:
  meupp-apex-intg -> ppapo: 16 times
  meitem-service -> mevend-user-profile: 14 times
  ppapl -> meupp-allowance: 14 times
  meitem-service -> mevend-vendor-api: 13 times
  meitem-batch -> meitem-service: 13 times
  mecost-notifications -> mevend-communication-service: 13 times
  meitem-workflow -> meitem-service: 12 times
  meitem-service -> meitem-common: 12 times
  meitem-service -> meitem-workflow: 12 times
  mevend-company -> memsp-sharedservice: 12 times

=== EXPECTED vs ACTUAL ===
Expected unique edges (no self-loops, no duplicates): 17123
Actual NetworkX graph edges: 17162
Difference: 39

⚠️ MISMATCH: Still 39 edges missing


In [22]:
# Show dependency example for a sample service
print("=== Service Dependency Example ===\n")

service_graph = graph_data_complete['service_graph']
service_features_cache = graph_data_complete['service_features_cache']

# Find a service with both upstream and downstream dependencies
sample_svc = None
for svc in service_graph.nodes():
    if service_graph.in_degree(svc) > 0 and service_graph.out_degree(svc) > 0:
        sample_svc = svc
        break

if sample_svc:
    print(f"Service: {sample_svc}")
    print(f"In-degree: {service_graph.in_degree(sample_svc)} (depends on these services)")
    print(f"Out-degree: {service_graph.out_degree(sample_svc)} (these services depend on it)")
    
    print(f"\nUpstream dependencies (services {sample_svc} depends on):")
    for predecessor in service_graph.predecessors(sample_svc):
        edge_data = service_graph.get_edge_data(predecessor, sample_svc)
        rel_type = edge_data.get('relationship_type', 'UNKNOWN') if edge_data else 'UNKNOWN'
        print(f"  ← {predecessor} ({rel_type})")
    
    print(f"\nDownstream dependencies (services that depend on {sample_svc}):")
    for successor in service_graph.successors(sample_svc):
        edge_data = service_graph.get_edge_data(sample_svc, successor)
        rel_type = edge_data.get('relationship_type', 'UNKNOWN') if edge_data else 'UNKNOWN'
        print(f"  → {successor} ({rel_type})")
    
    # Show the pre-computed features for this service
    print(f"\n\nPre-computed features for {sample_svc}:")
    svc_features = service_features_cache[sample_svc]
    print(f"  PageRank: {svc_features['pagerank']:.4f}")
    print(f"  Betweenness: {svc_features['betweenness']:.4f}")
    print(f"  Clustering Coef: {svc_features['clustering_coef']:.4f}")
    print(f"  Degree (total/in/out): {svc_features['degree_total']}/{svc_features['in_degree']}/{svc_features['out_degree']}")
    print(f"  Upstream/Downstream: {svc_features['num_upstream']}/{svc_features['num_downstream']}")
    print(f"  Dependency direction: {svc_features['dependency_direction']} ({'provider' if svc_features['dependency_direction'] > 0 else 'consumer'})")

=== Service Dependency Example ===

Service: ocmo-3pmtoorchestrator
In-degree: 9 (depends on these services)
Out-degree: 2 (these services depend on it)

Upstream dependencies (services ocmo-3pmtoorchestrator depends on):
  ← osfm-1p-pick-processor (CALLS)
  ← osco-payment-processor (CALLS)
  ← osco-picking-processor (CALLS)
  ← osco-processor (CALLS)
  ← osfm-order-create (CALLS)
  ← osos-order-status-webhook-service (CALLS)
  ← osfm-api-services (CALLS)
  ← osfm-release-processor (CALLS)
  ← osfm-lm-processor (CALLS)

Downstream dependencies (services that depend on ocmo-3pmtoorchestrator):
  → order_info (CALLS)
  → osfm-order-create (CALLS)


Pre-computed features for ocmo-3pmtoorchestrator:
  PageRank: 0.0002
  Betweenness: 0.0000
  Clustering Coef: 0.2222
  Degree (total/in/out): 11/9/2
  Upstream/Downstream: 9/2
  Dependency direction: -7 (consumer)


In [23]:
# Display top services by centrality metrics
print("=== Top Services by Graph Metrics ===\n")

pagerank_cache = graph_data_complete['pagerank_cache']
betweenness_cache = graph_data_complete['betweenness_cache']
clustering_coef_cache = graph_data_complete['clustering_coef_cache']

# Top by PageRank
top_pagerank = sorted(pagerank_cache.items(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 services by PageRank (most important):")
for service, score in top_pagerank:
    print(f"  {service}: {score:.4f}")

# Top by Betweenness
top_betweenness = sorted(betweenness_cache.items(), key=lambda x: x[1], reverse=True)[:5]
print("\nTop 5 services by Betweenness (hub/bridge services):")
for service, score in top_betweenness:
    print(f"  {service}: {score:.4f}")

# Top by Clustering Coefficient
top_clustering = sorted(clustering_coef_cache.items(), key=lambda x: x[1], reverse=True)[:5]
print("\nTop 5 services by Clustering Coefficient (tightly connected):")
for service, score in top_clustering:
    print(f"  {service}: {score:.4f}")

=== Top Services by Graph Metrics ===

Top 5 services by PageRank (most important):
  esco-aksba2-nonprod-westus-cluster-01: 0.0318
  esco-aksba1-nonprod-westus-cluster-01: 0.0142
  esco-aksshared-nonprod-westus-cluster-05: 0.0110
  dxinaz-springcloud: 0.0061
  esco-aksshared-nonprod-westus-cluster-01: 0.0046

Top 5 services by Betweenness (hub/bridge services):
  iauc-profilemanagement: 0.0000
  iauc-preferencemanagement: 0.0000
  iauc-householdmanagement: 0.0000
  iauc-session-management-service: 0.0000
  iauc-profile-state-api: 0.0000

Top 5 services by Clustering Coefficient (tightly connected):
  osos-order-status-service: 1.0000
  dxinaz-inventory-cosmos: 1.0000
  mevir-invoice-bk-azr: 1.0000
  dxinaz-recv-detail: 1.0000
  osdt-receipt-generator: 1.0000


In [24]:
# Display the service features cache from loaded graph
print("=== Service Features Cache ===\n")

service_features_cache = graph_data_complete['service_features_cache']

# Convert to DataFrame for better visualization
features_df = pd.DataFrame.from_dict(service_features_cache, orient='index')

print(f"Total services with features: {len(features_df)}")
print(f"Total features per service: {len(features_df.columns)}\n")

print("Feature columns:")
for i, col in enumerate(features_df.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\n\nSample services with their graph features:")
display(features_df.head(5))

=== Service Features Cache ===

Total services with features: 16064
Total features per service: 20

Feature columns:
   1. degree_total
   2. in_degree
   3. out_degree
   4. pagerank
   5. betweenness
   6. clustering_coef
   7. num_upstream
   8. num_downstream
   9. upstream_calls
  10. upstream_owns
  11. upstream_belongs_to
  12. downstream_calls
  13. downstream_owns
  14. downstream_belongs_to
  15. ratio_calls
  16. ratio_owns
  17. ratio_belongs_to
  18. dependency_direction
  19. avg_neighbor_degree
  20. max_neighbor_degree


Sample services with their graph features:


,degree_total,in_degree,out_degree,pagerank,betweenness,clustering_coef,num_upstream,num_downstream,upstream_calls,upstream_owns,upstream_belongs_to,downstream_calls,downstream_owns,downstream_belongs_to,ratio_calls,ratio_owns,ratio_belongs_to,dependency_direction,avg_neighbor_degree,max_neighbor_degree
ocpe-storecloudconfig,5,0,5,0.000055,0.0,0.000000,0,5,0,0,0,5,0,0,1.0,0.0,0.0,5,4.200000,11
ocmo-3pmtoorchestrator,11,9,2,0.000154,0.0,0.222222,9,2,9,0,0,2,0,0,1.0,0.0,0.0,-7,10.545455,23
menfpt-calendar-service,5,1,4,0.000063,0.0,0.400000,1,4,1,0,0,4,0,0,1.0,0.0,0.0,3,3.600000,6
menfpt_qa1,3,3,0,0.000091,0.0,0.333333,3,0,3,0,0,0,0,0,1.0,0.0,0.0,-3,5.333333,6
menfpt-forecast-service,6,0,6,0.000055,0.0,0.266667,0,6,0,0,0,6,0,0,1.0,0.0,0.0,6,3.000000,5


## Alert-to-Graph Mapping

Now we'll demonstrate how alerts are mapped to graph services using the enrichment method.

In [26]:
print
# maps alerts to graph services with primary and fallback strategies

def enrich_alert_with_graph_info(alert, service_to_graph, MIN_MATCH_SCORE=2):
    """Map alert to graph service using service_name, with fallback to node/namespace/cluster"""
    service_name = alert.get('service_name', '')
    
    # Primary mapping: direct service_name match
    if service_name and service_name in service_to_graph:
        alert['graph_service'] = service_name
        alert['graph_info'] = service_to_graph[service_name]
        alert['mapping_method'] = 'service_name'
        alert['mapping_confidence'] = 1.0
        return True
    
    # Fallback mapping: match by namespace + cluster combination
    alert_namespace = alert.get('namespace', '')
    alert_cluster = alert.get('cluster', '')
    alert_node = alert.get('node', '')
    
    # find matching services by namespace and cluster
    matched_services = []
    for svc_name, svc_info in service_to_graph.items():
        match_score = 0
        
        # Match namespace
        if alert_namespace and svc_info.get('namespace') == alert_namespace:
            match_score += 2
        
        # Match cluster
        if alert_cluster and svc_info.get('cluster') == alert_cluster:
            match_score += 2
        
        # Match node (if present in service properties)
        if alert_node and svc_info.get('properties', {}).get('node') == alert_node:
            match_score += 1
        
        if match_score >= MIN_MATCH_SCORE:
            matched_services.append((svc_name, svc_info, match_score))
    
    # Use best match if found
    if matched_services:
        matched_services.sort(key=lambda x: x[2], reverse=True)
        best_match = matched_services[0]
        
        alert['graph_service'] = best_match[0]
        alert['graph_info'] = best_match[1]
        alert['mapping_method'] = 'namespace_cluster_fallback'
        alert['match_score'] = best_match[2]
        alert['mapping_confidence'] = best_match[2] / 5.0
        return True
    
    # No mapping found
    alert['graph_service'] = None
    alert['graph_info'] = None
    alert['mapping_method'] = 'unmapped'
    alert['mapping_confidence'] = 0.0
    return False

# Enrich first 3 alerts
enriched_alerts = []
for alert in firing_alerts[:3]:
    alert_copy = alert.copy()
    enrich_alert_with_graph_info(alert_copy, service_to_graph)
    enriched_alerts.append(alert_copy)
    
# Display mapping results
print("Alert-to-Graph Mapping Results:")
print("=" * 80)
for i, alert in enumerate(enriched_alerts):
    print(f"\nAlert {i+1}:")
    print(f"  Service Name: {alert.get('service_name', 'N/A')}")
    print(f"  Graph Service: {alert.get('graph_service', 'N/A')}")
    print(f"  Mapping Method: {alert.get('mapping_method', 'N/A')}")
    print(f"  Mapping Confidence: {alert.get('mapping_confidence', 0):.2f}")
    print(f"  Namespace: {alert.get('namespace', 'N/A')}")
    print(f"  Cluster: {alert.get('cluster', 'N/A')}")

Alert-to-Graph Mapping Results:

Alert 1:
  Service Name: 
  Graph Service: None
  Mapping Method: unmapped
  Mapping Confidence: 0.00
  Namespace: 
  Cluster: esco-aksres-nonprod-westus-cluster-01

Alert 2:
  Service Name: 
  Graph Service: None
  Mapping Method: unmapped
  Mapping Confidence: 0.00
  Namespace: 
  Cluster: esco-aksba1-nonprod-westus-cluster-01

Alert 3:
  Service Name: 
  Graph Service: None
  Mapping Method: unmapped
  Mapping Confidence: 0.00
  Namespace: 
  Cluster: esco-aksba1-nonprod-westus-cluster-01

Alert 4:
  Service Name: 
  Graph Service: None
  Mapping Method: unmapped
  Mapping Confidence: 0.00
  Namespace: 
  Cluster: esco-aksba1-nonprod-westus-cluster-01

Alert 5:
  Service Name: 
  Graph Service: None
  Mapping Method: unmapped
  Mapping Confidence: 0.00
  Namespace: 
  Cluster: esco-aksba1-nonprod-westus-cluster-01

Alert 6:
  Service Name: 
  Graph Service: None
  Mapping Method: unmapped
  Mapping Confidence: 0.00
  Namespace: 
  Cluster: esco-aksba

## Get Service Dependencies

retrieve upstream and downstream dependencies for a service.

In [27]:

def get_service_dependencies(service_name, service_graph):
    """Get upstream and downstream dependencies for a service"""
    dependencies = {
        'upstream': [],
        'downstream': [],
        'peers': []
    }
    
    if not service_graph.has_node(service_name):
        return dependencies
    
    # Get upstream dependencies
    for predecessor in service_graph.predecessors(service_name):
        edge_data = service_graph.get_edge_data(predecessor, service_name)
        dependencies['upstream'].append({
            'service': predecessor,
            'relationship': edge_data.get('relationship_type', '') if edge_data else ''
        })
    
    # Get downstream dependencies
    for successor in service_graph.successors(service_name):
        edge_data = service_graph.get_edge_data(service_name, successor)
        dependencies['downstream'].append({
            'service': successor,
            'relationship': edge_data.get('relationship_type', '') if edge_data else ''
        })
    
    return dependencies

# dependencies for top 3 services by PageRank
print("Service Dependencies :")
print("=" * 80)

for service, _ in top_pagerank[:3]:
    deps = get_service_dependencies(service, service_graph)
    print(f"\nService: {service}")
    print(f"  Upstream ({len(deps['upstream'])} services):")
    for dep in deps['upstream'][:5]:  # Show first 5
        print(f"    <- {dep['service']} ({dep['relationship']})")
    print(f"  Downstream ({len(deps['downstream'])} services):")
    for dep in deps['downstream'][:5]:  # Show first 5
        print(f"    -> {dep['service']} ({dep['relationship']})")

Service Dependencies :

Service: esco-aksba2-nonprod-westus-cluster-01
  Upstream (685 services):
    <- afap-dev (BELONGS_TO)
    <- akv2k8s (BELONGS_TO)
    <- appdynamics (BELONGS_TO)
    <- calico-system (BELONGS_TO)
    <- escf-dev1 (BELONGS_TO)
  Downstream (0 services):

Service: esco-aksba1-nonprod-westus-cluster-01
  Upstream (314 services):
    <- aais-perf1 (BELONGS_TO)
    <- aapn-perf1 (BELONGS_TO)
    <- afap-dev (BELONGS_TO)
    <- akv2k8s (BELONGS_TO)
    <- appdynamics (BELONGS_TO)
  Downstream (0 services):

Service: esco-aksshared-nonprod-westus-cluster-05
  Upstream (243 services):
    <- afap-dev (BELONGS_TO)
    <- akv2k8s (BELONGS_TO)
    <- calico-system (BELONGS_TO)
    <- escf-dev1 (BELONGS_TO)
    <- esob-perf1 (BELONGS_TO)
  Downstream (0 services):


## Transitive Parent Services

finds all upstream services (parents) up to a specified depth using BFS traversal.

In [28]:

def get_transitive_parent_services(service_name, service_graph, max_depth=3):
    parent_set = set()
    
    if not service_graph.has_node(service_name):
        return parent_set
    
    # BFS to collect upstream services
    visited = set()
    queue = [(service_name, 0)]  # (service, depth)
    
    while queue:
        current, depth = queue.pop(0)
        visited.add(current)
        
        if depth >= max_depth:
            continue
        
        # Get immediate predecessors (parents)
        for predecessor in service_graph.predecessors(current):
            if predecessor not in visited and predecessor != service_name:
                parent_set.add(predecessor)
                queue.append((predecessor, depth + 1))
    
    return parent_set

# Find transitive parents for top services
print("Transitive Parent Services Demo:")
print("=" * 80)

for service, _ in top_pagerank[:3]:
    parents = get_transitive_parent_services(service, service_graph, max_depth=3)
    print(f"\nService: {service}")
    print(f"  Transitive Parents (depth=3): {len(parents)} services")
    if parents:
        print(f"  Examples: {', '.join(list(parents)[:5])}")

Transitive Parent Services Demo:

Service: esco-aksba2-nonprod-westus-cluster-01
  Transitive Parents (depth=3): 725 services
  Examples: ocim-stage, emcn-dev, emju-perf1, wcax-stage, flot-dev

Service: esco-aksba1-nonprod-westus-cluster-01
  Transitive Parents (depth=3): 314 services
  Examples: aks-escoba1np02-13277147-vmss000438, mepl-dev, aks-escoba1np01-28933280-vmss0007h0, akv2k8s, aks-escoba1np01-28933280-vmss0003u4

Service: esco-aksshared-nonprod-westus-cluster-05
  Transitive Parents (depth=3): 243 services
  Examples: aks-escoshdnp02-77652852-vmss0000dq, aks-escoshdnp03-21905038-vmss00004b, aks-escoshdnp04-39608561-vmss00000e, aks-escoshdnp02-77652852-vmss0000d4, aks-escoshdnp02-77652852-vmss0000fb


## Compute Path Similarity

Calculate similarity between two services based on their graph traversal paths using Jaccard similarity.

In [29]:

def compute_path_similarity(service1, service2, service_graph):
    if service1 == service2:
        return 1.0
    
    # Get transitive upstream sinks for both services
    parents1 = get_transitive_parent_services(service1, service_graph)
    parents2 = get_transitive_parent_services(service2, service_graph)
    
    if not parents1 or not parents2:
        return 0.0
    
    # Jaccard similarity of parent sets
    intersection = len(parents1 & parents2)
    union = len(parents1 | parents2)
    
    if union == 0:
        return 0.0
    
    return intersection / union

# Compute path similarity between services
print("Path Similarity Demo:")
print("=" * 80)

# Compare top 5 services pairwise
top_services = [svc for svc, _ in top_pagerank[:5]]

print("\nPairwise Path Similarity Matrix:")
print(f"{'Service':<25}", end="")
for svc in top_services:
    print(f"{svc[:15]:<17}", end="")
print()

for svc1 in top_services:
    print(f"{svc1[:25]:<25}", end="")
    for svc2 in top_services:
        similarity = compute_path_similarity(svc1, svc2, service_graph)
        print(f"{similarity:>17.3f}", end="")
    print()

# Find most similar pair
print("\nMost Similar Service Pairs:")
similarities = []
for i, svc1 in enumerate(top_services):
    for j, svc2 in enumerate(top_services):
        if i < j:
            sim = compute_path_similarity(svc1, svc2, service_graph)
            if sim > 0:
                similarities.append((svc1, svc2, sim))

similarities.sort(key=lambda x: x[2], reverse=True)
for svc1, svc2, sim in similarities[:5]:
    print(f"  {svc1} <-> {svc2}: {sim:.3f}")

Path Similarity Demo:

Pairwise Path Similarity Matrix:
Service                  esco-aksba2-non  esco-aksba1-non  esco-aksshared-  dxinaz-springcl  esco-aksshared-  
esco-aksba2-nonprod-westu            1.000            0.012            0.008            0.000            0.015
esco-aksba1-nonprod-westu            0.012            1.000            0.013            0.000            0.027
esco-aksshared-nonprod-we            0.008            0.013            1.000            0.000            0.023
dxinaz-springcloud                   0.000            0.000            0.000            1.000            0.000
esco-aksshared-nonprod-we            0.015            0.027            0.023            0.000            1.000

Most Similar Service Pairs:
  esco-aksba1-nonprod-westus-cluster-01 <-> esco-aksshared-nonprod-westus-cluster-01: 0.027
  esco-aksshared-nonprod-westus-cluster-05 <-> esco-aksshared-nonprod-westus-cluster-01: 0.023
  esco-aksba2-nonprod-westus-cluster-01 <-> esco-aksshared-non

Feature Engineering - Severity & Category Encoding

alert metadata is encoded into numerical features for machine learning.

In [30]:
# Feature encoding functions
def encode_severity(severity):
    """Encode severity to numeric value"""
    severity_map = {
        'critical': 4,
        'high': 3,
        'warning': 2,
        'info': 1,
        'unknown': 0
    }
    return severity_map.get(str(severity).lower().strip(), 0)

def encode_alert_category(category):
    """Encode alert category to numeric value"""
    category_map = {
        'saturation': 1,
        'anomaly': 2,
        'error': 3,
        'critical': 4,
        'failure': 5,
        'slo': 6,
        'unknown': 0
    }
    return category_map.get(str(category).lower().strip(), 0)

def encode_alert_subcategory(subcategory):
    """Encode alert subcategory to numeric value"""
    subcategory_map = {
        'hpa': 1,
        'resource': 2,
        'error': 3,
        'node': 4,
        'memory': 5,
        'latency': 6,
        'other': 7,
        'volume': 8,
        'cpu': 9,
        'unknown': 0
    }
    return subcategory_map.get(str(subcategory).lower().strip(), 0)

def encode_workload_type(workload_type):
    """Encode workload type to numeric value"""
    workload_map = {
        'deployment': 1,
        'daemonset': 2,
        'statefulset': 3,
        'job': 4,
        'cronjob': 5,
        'pod': 6,
        'unknown': 0
    }
    return workload_map.get(str(workload_type).lower().strip(), 0)

# Encode features for sample alerts
print("Feature Encoding Demo:")
print("=" * 80)

sample_alerts = firing_alerts[:5]
encoded_features = []

for alert in sample_alerts:
    features = {
        'alert_name': alert.get('alert_name', ''),
        'severity': alert.get('severity', ''),
        'severity_encoded': encode_severity(alert.get('severity', '')),
        'category': alert.get('alert_category', ''),
        'category_encoded': encode_alert_category(alert.get('alert_category', '')),
        'subcategory': alert.get('alert_subcategory', ''),
        'subcategory_encoded': encode_alert_subcategory(alert.get('alert_subcategory', '')),
        'workload': alert.get('workload_type', ''),
        'workload_encoded': encode_workload_type(alert.get('workload_type', ''))
    }
    encoded_features.append(features)

# Display as DataFrame
import pandas as pd
encoded_df = pd.DataFrame(encoded_features)
print("\nEncoded Features for Sample Alerts:")
display(encoded_df[['alert_name', 'severity', 'severity_encoded', 
                    'category', 'category_encoded', 
                    'subcategory', 'subcategory_encoded']])

Feature Encoding Demo:

Encoded Features for Sample Alerts:


,alert_name,severity,severity_encoded,category,category_encoded,subcategory,subcategory_encoded
0,ResourceRateAnomaly,warning,2,anomaly,2,Resource,2
1,ResourceRateAnomaly,warning,2,anomaly,2,Resource,2
2,ResourceRateAnomaly,warning,2,anomaly,2,Resource,2
3,ResourceRateAnomaly,warning,2,anomaly,2,Resource,2
4,ResourceRateAnomaly,warning,2,anomaly,2,Resource,2


##  data classification feature Engineering

39 features (20 graph + 19 alert metadata) for an alert.

In [31]:
# feature extraction for an alert
def extract_complete_features(alert, service_features_cache):
    """Extract all 39 features for an alert (20 graph + 19 metadata)"""
    
    graph_feature_keys = ['degree_total', 'in_degree', 'out_degree', 'pagerank', 'betweenness', 
                         'clustering_coef', 'num_upstream', 'num_downstream',
                         'upstream_calls', 'upstream_owns', 'upstream_belongs_to',
                         'downstream_calls', 'downstream_owns', 'downstream_belongs_to',
                         'ratio_calls', 'ratio_owns', 'ratio_belongs_to',
                         'dependency_direction', 'avg_neighbor_degree', 'max_neighbor_degree']
    
    feature_dict = {}
    
    # Graph features (20 features)
    graph_service = alert.get('graph_service', '')
    if graph_service and graph_service in service_features_cache:
        cached_features = service_features_cache[graph_service]
        for key in graph_feature_keys:
            feature_dict[key] = cached_features.get(key, 0)
    else:
        for key in graph_feature_keys:
            feature_dict[key] = 0
    
    # Alert metadata features (19 features)
    alert_name = str(alert.get('alert_name', '')).lower()
    feature_dict['severity_encoded'] = encode_severity(alert.get('severity', ''))
    feature_dict['is_error_alert'] = 1 if 'error' in alert_name else 0
    feature_dict['is_resource_alert'] = 1 if any(x in alert_name for x in ['cpu', 'memory', 'hpa', 'resource']) else 0
    feature_dict['is_network_alert'] = 1 if any(x in alert_name for x in ['network', 'rx_bytes', 'tx_bytes']) else 0
    feature_dict['is_anomaly_alert'] = 1 if 'anomaly' in str(alert.get('alert_category', '')).lower() else 0
    feature_dict['alert_category_encoded'] = encode_alert_category(alert.get('alert_category', ''))
    feature_dict['alert_subcategory_encoded'] = encode_alert_subcategory(alert.get('alert_subcategory', ''))
    feature_dict['workload_type_encoded'] = encode_workload_type(alert.get('workload_type', ''))
    
    # Temporal features
    start_dt = alert.get('start_datetime')
    if start_dt and pd.notna(start_dt):
        feature_dict['hour_of_day'] = start_dt.hour
        feature_dict['day_of_week'] = start_dt.dayofweek
        feature_dict['is_business_hours'] = 1 if 9 <= start_dt.hour <= 17 else 0
        feature_dict['is_weekend'] = 1 if start_dt.dayofweek >= 5 else 0
    else:
        feature_dict['hour_of_day'] = 0
        feature_dict['day_of_week'] = 0
        feature_dict['is_business_hours'] = 0
        feature_dict['is_weekend'] = 0
    
    # Category-Subcategory combination features
    category = str(alert.get('alert_category', '')).lower().strip()
    subcategory = str(alert.get('alert_subcategory', '')).lower().strip()
    
    feature_dict['is_critical_resource'] = 1 if (category in ['critical', 'failure'] and subcategory == 'resource') else 0
    feature_dict['is_saturation_memory'] = 1 if (category == 'saturation' and subcategory == 'memory') else 0
    feature_dict['is_saturation_cpu'] = 1 if (category == 'saturation' and subcategory == 'cpu') else 0
    feature_dict['is_error_node'] = 1 if (category == 'error' and subcategory == 'node') else 0
    feature_dict['is_anomaly_latency'] = 1 if (category == 'anomaly' and subcategory == 'latency') else 0
    feature_dict['is_slo_violation'] = 1 if category == 'slo' else 0
    feature_dict['mapping_confidence'] = alert.get('mapping_confidence', 0)
    
    return feature_dict

# Extract complete features for sample alert
sample_alert = firing_alerts[0].copy()
enrich_alert_with_graph_info(sample_alert, service_to_graph)

features = extract_complete_features(sample_alert, service_features_cache)

print("Complete Feature Extraction Demo:")
print("=" * 80)
print(f"\nAlert: {sample_alert.get('alert_name', 'N/A')}")
print(f"Service: {sample_alert.get('graph_service', 'N/A')}")
print(f"\nTotal Features: {len(features)}")
print("\nGraph Features (20):")
for i, (key, val) in enumerate(list(features.items())[:20]):
    print(f"  {key:25s} = {val:8.4f}")
print("\nAlert Metadata Features (19):")
for i, (key, val) in enumerate(list(features.items())[20:]):
    print(f"  {key:25s} = {val:8.4f}")

Complete Feature Extraction Demo:

Alert: ResourceRateAnomaly
Service: None

Total Features: 39

Graph Features (20):
  degree_total              =   0.0000
  in_degree                 =   0.0000
  out_degree                =   0.0000
  pagerank                  =   0.0000
  betweenness               =   0.0000
  clustering_coef           =   0.0000
  num_upstream              =   0.0000
  num_downstream            =   0.0000
  upstream_calls            =   0.0000
  upstream_owns             =   0.0000
  upstream_belongs_to       =   0.0000
  downstream_calls          =   0.0000
  downstream_owns           =   0.0000
  downstream_belongs_to     =   0.0000
  ratio_calls               =   0.0000
  ratio_owns                =   0.0000
  ratio_belongs_to          =   0.0000
  dependency_direction      =   0.0000
  avg_neighbor_degree       =   0.0000
  max_neighbor_degree       =   0.0000

Alert Metadata Features (19):
  severity_encoded          =   2.0000
  is_error_alert            =   

## Alert Duplicate Detection

Demonstrate the duplicate detection algorithm using multiple similarity metrics.

In [32]:

def are_duplicates(alert1, alert2, time_window_minutes=5, duplicate_threshold=5, 
                   description_similarity_threshold=0.7):
    """
    Determine if two alerts are duplicates based on multiple similarity metrics.
    
    Scoring system:
    - Same alert name: +2 points
    - Same service: +2 points  
    - Same severity: +1 point
    - Similar description: +1 point (if Jaccard > threshold)
    - Within time window: +1 point
    
    Threshold: >= 5 points = duplicates
    """
    score = 0
    
    # Same alert name
    if alert1.get('alert_name', '') == alert2.get('alert_name', ''):
        score += 2
    
    # Same service
    if alert1.get('graph_service', '') == alert2.get('graph_service', ''):
        score += 2
    
    # Same severity
    if alert1.get('severity', '') == alert2.get('severity', ''):
        score += 1
    
    # Similar description (Jaccard similarity)
    desc1 = set(str(alert1.get('description', '')).lower().split())
    desc2 = set(str(alert2.get('description', '')).lower().split())
    
    if desc1 and desc2:
        intersection = len(desc1 & desc2)
        union = len(desc1 | desc2)
        if union > 0:
            jaccard = intersection / union
            if jaccard >= description_similarity_threshold:
                score += 1
    
    # Within time window
    time1 = alert1.get('start_timestamp', 0)
    time2 = alert2.get('start_timestamp', 0)
    
    if time1 and time2:
        time_diff_minutes = abs(time1 - time2) / 60
        if time_diff_minutes <= time_window_minutes:
            score += 1
    
    return score >= duplicate_threshold, score

# Check for duplicates in sample alerts
print("Duplicate Detection Demo:")
print("=" * 80)

# Find some alerts with same service for testing
service_alerts = {}
for alert in firing_alerts[:50]:
    svc = alert.get('service_name', 'unknown')
    if svc not in service_alerts:
        service_alerts[svc] = []
    service_alerts[svc].append(alert)

# Check for duplicates within same service
print("\nChecking for duplicates within services:")
duplicate_count = 0

for service, alerts in list(service_alerts.items())[:5]:
    if len(alerts) < 2:
        continue
    
    print(f"\nService: {service} ({len(alerts)} alerts)")
    
    for i in range(len(alerts)):
        for j in range(i+1, len(alerts)):
            is_dup, score = are_duplicates(alerts[i], alerts[j])
            if score >= 3:  # Show potential duplicates
                print(f"  Alert {i+1} vs Alert {j+1}:")
                print(f"    Score: {score}/7")
                print(f"    Is Duplicate: {is_dup}")
                print(f"    Alert 1: {alerts[i].get('alert_name', 'N/A')[:50]}")
                print(f"    Alert 2: {alerts[j].get('alert_name', 'N/A')[:50]}")
                if is_dup:
                    duplicate_count += 1

print(f"\nTotal duplicates found: {duplicate_count}")

Duplicate Detection Demo:

Checking for duplicates within services:

Service:  (20 alerts)
  Alert 1 vs Alert 2:
    Score: 7/7
    Is Duplicate: True
    Alert 1: ResourceRateAnomaly
    Alert 2: ResourceRateAnomaly
  Alert 1 vs Alert 3:
    Score: 6/7
    Is Duplicate: True
    Alert 1: ResourceRateAnomaly
    Alert 2: ResourceRateAnomaly
  Alert 1 vs Alert 4:
    Score: 6/7
    Is Duplicate: True
    Alert 1: ResourceRateAnomaly
    Alert 2: ResourceRateAnomaly
  Alert 1 vs Alert 5:
    Score: 6/7
    Is Duplicate: True
    Alert 1: ResourceRateAnomaly
    Alert 2: ResourceRateAnomaly
  Alert 1 vs Alert 6:
    Score: 6/7
    Is Duplicate: True
    Alert 1: ResourceRateAnomaly
    Alert 2: ResourceRateAnomaly
  Alert 1 vs Alert 7:
    Score: 6/7
    Is Duplicate: True
    Alert 1: ResourceRateAnomaly
    Alert 2: ResourceRateAnomaly
  Alert 1 vs Alert 8:
    Score: 6/7
    Is Duplicate: True
    Alert 1: ResourceRateAnomaly
    Alert 2: ResourceRateAnomaly
  Alert 1 vs Alert 9:
    S

## Summary Statistics

end2end pipeline summary showing how alerts are processed and consolidated.

In [34]:
# Complete Pipeline Summary
print("Alert Consolidation Pipeline Summary")
print("=" * 80)

# Data Loading
print("\n[PHASE 1] Data Loading & Preprocessing")
print(f"  Total Firing Alerts: {len(firing_alerts)}")
print(f"  Unique Services: {len(set(a.get('service_name', '') for a in firing_alerts))}")
print(f"  Unique Namespaces: {len(set(a.get('namespace', '') for a in firing_alerts))}")
print(f"  Unique Clusters: {len(set(a.get('cluster', '') for a in firing_alerts))}")

# Graph Data
print("\n[PHASE 2] Service Graph")
print(f"  Graph Services: {len(service_to_graph)}")
print(f"  Graph Edges: {service_graph.number_of_edges()}")
print(f"  Pre-computed Features per Service: 20")

# Alert-to-Graph Mapping
enriched_count = 0
mapped_count = 0
unmapped_count = 0

for alert in firing_alerts[:100]:  # Sample first 100
    alert_copy = alert.copy()
    result = enrich_alert_with_graph_info(alert_copy, service_to_graph)
    enriched_count += 1
    if result:
        mapped_count += 1
    else:
        unmapped_count += 1

mapping_rate = (mapped_count / enriched_count * 100) if enriched_count > 0 else 0

print("\n[PHASE 3] Alert-to-Graph Mapping (sample of 100)")
print(f"  Enriched Alerts: {enriched_count}")
print(f"  Successfully Mapped: {mapped_count} ({mapping_rate:.1f}%)")
print(f"  Unmapped: {unmapped_count}")

# Phase 4: Feature Engineering
print("\n[PHASE 4] Feature Engineering")
print(f"  Total Features: 39")
print(f"    - Graph Topology Features: 20")
print(f"    - Alert Metadata Features: 19")
print(f"  Feature Categories:")
print(f"    - Degree metrics: 3")
print(f"    - Centrality metrics: 3")
print(f"    - Relationship counts: 6")
print(f"    - Relationship ratios: 3")
print(f"    - Dependency metrics: 3")
print(f"    - Alert severity/category: 4")
print(f"    - Alert type flags: 4")
print(f"    - Temporal features: 4")
print(f"    - Category combinations: 6")
print(f"    - Mapping confidence: 1")

# Service Relationships
print("\n[PHASE 5] Service Relationship Analysis")
# Get sample dependency stats
sample_services = list(service_graph.nodes())[:10]
avg_upstream = sum(service_graph.in_degree(s) for s in sample_services) / len(sample_services)
avg_downstream = sum(service_graph.out_degree(s) for s in sample_services) / len(sample_services)

print(f"  Average Upstream Dependencies: {avg_upstream:.1f}")
print(f"  Average Downstream Dependencies: {avg_downstream:.1f}")
print(f"  Path Similarity Threshold: 0.3 (30% shared parents)")

# Deduplication
print("\n[PHASE 6] Duplicate Detection")
print(f"  Time Window: 5 minutes")
print(f"  Duplicate Threshold: 5/7 points")
print(f"  Similarity Metrics:")
print(f"    - Alert name match: 2 points")
print(f"    - Service match: 2 points")
print(f"    - Severity match: 1 point")
print(f"    - Description similarity: 1 point")
print(f"    - Time window: 1 point")

print("\n" + "=" * 80)
print("Tutorial Complete - All Methods Demonstrated!")

Alert Consolidation Pipeline Summary

[PHASE 1] Data Loading & Preprocessing
  Total Firing Alerts: 13960
  Unique Services: 871
  Unique Namespaces: 411
  Unique Clusters: 21

[PHASE 2] Service Graph
  Graph Services: 0
  Graph Edges: 17162
  Pre-computed Features per Service: 20

[PHASE 3] Alert-to-Graph Mapping (sample of 100)
  Enriched Alerts: 100
  Successfully Mapped: 0 (0.0%)
  Unmapped: 100

[PHASE 4] Feature Engineering
  Total Features: 39
    - Graph Topology Features: 20
    - Alert Metadata Features: 19
  Feature Categories:
    - Degree metrics: 3
    - Centrality metrics: 3
    - Relationship counts: 6
    - Relationship ratios: 3
    - Dependency metrics: 3
    - Alert severity/category: 4
    - Alert type flags: 4
    - Temporal features: 4
    - Category combinations: 6
    - Mapping confidence: 1

[PHASE 5] Service Relationship Analysis
  Average Upstream Dependencies: 1.8
  Average Downstream Dependencies: 4.5
  Path Similarity Threshold: 0.3 (30% shared parents)

